In [ ]:
!pip install sentence-transformers faiss-cpu 

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install faiss-cpu 
!pip install --upgrade transformers

In [ ]:
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the FAISS index from the file
index_file_path = "/lakehouse/default/Files/movie_plot_miniLM.index"
index = faiss.read_index(index_file_path)

# Load the IMDb movie plots into a Pandas DataFrame (you can load it from SQL database as well)
imdb_plots_df = pd.read_csv("/lakehouse/default/" + "Files/movie_plots.csv")

# Load the pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Function to convert text to vector and normalize
def text_to_vector(text):
    vector = model.encode([text], convert_to_numpy=True)
    return vector / np.linalg.norm(vector, axis=1, keepdims=True)

# Function to perform the search and fetch the top N closest results
def search_faiss_index(query_text, top_n=5):
    # Convert the query text to a vector
    query_vector = text_to_vector(query_text)
    
    # Ensure the query vector is in the correct shape
    query_vector = np.array(query_vector, dtype='float32')
    
    # Perform the search
    distances, indices = index.search(query_vector, top_n)
    
    # Fetch the corresponding movie titles and plots from the DataFrame
    closest_plots = imdb_plots_df.iloc[indices[0]][['Title', 'Plot']]
    
    return closest_plots

In [18]:
query_text = "A high-stakes heist in a bustling city"  
top_n = 5
closest_movies = search_faiss_index(query_text, top_n)

print(closest_movies)

StatementMeta(, 44b0f98c-1eea-4e7a-be16-0a39b1217d81, 30, Finished, Available, Finished)

                      Title                                               Plot
19139           The Diamond  After a gang pulls off a heist to acquire fres...
6400   Six Bridges to Cross  Jerry Florea (Tony Curtis) is planning a heist...
8748                Villain  Ruthless East End gangster Vic Dakin has plans...
16498          Empire State  After failing to get into the local police aca...
4475              High Tide  Newspaperman is caught in the middle of a ruth...
